# Expanded Image Analysis Lab

## Instructions
Using one of these packages:
1. imager 
2. EBImage from bioc 
3. ripa 
4. adimpro

perform an image processing task and then analysis task.

Options are:

[tasks here]

With one or more of these images  (or use your own):

[image choices from web and sample images folder]

